In [393]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import random

In [394]:
data = pd.read_csv('dataset/dataCovidIndonesia.csv')
data = data.loc[0:49, 'Kasus harian']
data = np.genfromtxt(data, dtype= None)
data

array([  2,   0,   0,   0,   2,   0,   2,  13,   8,   0,  35,  27,  21,
        17,  38,  55,  82,  60,  81,  64,  65, 106, 105, 103, 153, 109,
       130, 129, 114, 149, 113, 196, 106, 181, 218, 247, 218, 337, 219,
       330, 399, 316, 282, 297, 380, 407, 325, 327, 185, 375])

In [395]:
def generasi_awal(pop_size, gene_per_pop):
    population_size = (pop_size, gene_per_pop)
    gene = np.random.uniform(low=0.1, high=0.9, size=population_size)
    return gene #generate individu dengan 3 gen pada setiap individu

In [396]:
def day_predict(pop_size, gene_per_pop):
    day = []
    for i in range(2, pop_size):
        row = []
        for j in range(0, gene_per_pop):
            if j == 0:
                row.append(data[i-1]) #y(t-1) 
            elif j == 1:
                row.append(data[i-2]) #y(t-2)
            elif j == 2:
                row.append(data[i]) #y(target)
        day.append(row)
    return day

In [397]:
def y_pred(pop_size, gene, day_predict, gene_per_pop):
    y_pred_indv = []
    for i in range (0,pop_size-2):
        row = []
        for j in gene:
            total = 0
            z = 0
            for k in j:
                if k<gene_per_pop-1:
                    g = k*day_predict[i][z]
                else:
                    g = k
                z=z+1
                total = total + g
            row.append(total)
        y_pred_indv.append(row)
    return y_pred_indv #generate y_pred_individu

In [398]:
def fitness_func(pop_size, day_predict, hasil_prediksi, gene_per_pop):  
    mape = []
    fitn = []
    for i in range(0,pop_size):
        row = []
        for j in range(0,len(data_y)):
            if hasil_prediksi[j][i]>0: #hitung mape
                row.append(abs((hasil_prediksi[j][i]-data_y[j])/hasil_prediksi[j][i]))
            else:
                row.append(abs(0)) #mape akan bernilai 0 ketika hasil_prediksi bernilai 0
        mape.append(row)        
    for i in range(0,len(mape)):
        f = 0
        for j in range (0,pop_size-2):
            f = f + mape[i][j]
        f=f/len(mape)
        fitn.append(1/(f+0.01))
    
    return fitn

In [399]:
def probability(fitness):
    prob = []
    sumF = 0
    for i in range(0,len(fitness)):
        sumF = sumF + fitness[i]
    for i in range(0,len(fitness)):
        prob.append(fitness[i]/sumF*100)
    return prob #generate probabilitas setiap fitness individu

In [400]:
def rangePc(probs):
    ranges = []
    x = 0
    y = 0
    minArray = []
    maxArray = []
    for i in range(0,len(probs)):
        if i == 0:
            x = 0;
        else:
            x = x + probs[i-1]
        y = y + probs[i]
        minArray.append(x)
        maxArray.append(y)
    ranges = np.vstack((minArray, maxArray)).T
    return ranges #generate nilai min dan max pada setiap probabilitas indifidu

In [401]:
def round_wheel(pc,fitness,ranges):
    rW = []
    for i in range(0,int(pc*50)):
        rNum = random.uniform(0,100)
        for j in range(0,len(ranges)):
            if rNum >= ranges[j][0] and rNum <= ranges[j][1]:
                rW.append(j)
    return rW #terpilih individu ke-sekian

In [402]:
pop_size = 50
gene_per_pop = 3
pc = 0.7
generasiAwal = generasi_awal(pop_size, gene_per_pop)
generasiAwal

array([[0.50334349, 0.80162763, 0.78117193],
       [0.82478407, 0.53435536, 0.54294391],
       [0.50082692, 0.49720653, 0.41535772],
       [0.71968277, 0.12986178, 0.414362  ],
       [0.60471336, 0.43680289, 0.43603772],
       [0.2504847 , 0.27628418, 0.75382691],
       [0.46102433, 0.62621538, 0.31599998],
       [0.74754678, 0.59345905, 0.59508164],
       [0.16835606, 0.56147322, 0.72939995],
       [0.47187427, 0.24020839, 0.38742048],
       [0.36225533, 0.55257529, 0.34124598],
       [0.41167065, 0.75626305, 0.30314497],
       [0.8879384 , 0.3649605 , 0.23691287],
       [0.44401759, 0.44634292, 0.18064252],
       [0.3680224 , 0.5702209 , 0.67928065],
       [0.27159136, 0.48477786, 0.47758085],
       [0.39042135, 0.71113383, 0.72997101],
       [0.44576288, 0.42052005, 0.77929585],
       [0.6083144 , 0.20410589, 0.11106499],
       [0.68015231, 0.15878353, 0.67151232],
       [0.6770407 , 0.72596271, 0.14984544],
       [0.32038291, 0.57340143, 0.16029431],
       [0.

In [403]:
dayPredict = day_predict(pop_size, gene_per_pop)
dayPredict

[[0, 2, 0],
 [0, 0, 0],
 [0, 0, 2],
 [2, 0, 0],
 [0, 2, 2],
 [2, 0, 13],
 [13, 2, 8],
 [8, 13, 0],
 [0, 8, 35],
 [35, 0, 27],
 [27, 35, 21],
 [21, 27, 17],
 [17, 21, 38],
 [38, 17, 55],
 [55, 38, 82],
 [82, 55, 60],
 [60, 82, 81],
 [81, 60, 64],
 [64, 81, 65],
 [65, 64, 106],
 [106, 65, 105],
 [105, 106, 103],
 [103, 105, 153],
 [153, 103, 109],
 [109, 153, 130],
 [130, 109, 129],
 [129, 130, 114],
 [114, 129, 149],
 [149, 114, 113],
 [113, 149, 196],
 [196, 113, 106],
 [106, 196, 181],
 [181, 106, 218],
 [218, 181, 247],
 [247, 218, 218],
 [218, 247, 337],
 [337, 218, 219],
 [219, 337, 330],
 [330, 219, 399],
 [399, 330, 316],
 [316, 399, 282],
 [282, 316, 297],
 [297, 282, 380],
 [380, 297, 407],
 [407, 380, 325],
 [325, 407, 327],
 [327, 325, 185],
 [185, 327, 375]]

In [404]:
yPred = y_pred(pop_size, generasiAwal, dayPredict, gene_per_pop)
yPred

[[1.6032552593449472,
  1.0687107137619403,
  0.9944130623260024,
  0.2597235563115179,
  0.8736057880217036,
  0.5525683643219754,
  1.2524307535870944,
  1.1869180959690075,
  1.1229464446887678,
  0.4804167723419857,
  1.1051505775111596,
  1.512526097789985,
  0.7299209917594462,
  0.8926858364652481,
  1.14044180784009,
  0.9695557236117007,
  1.422267667165359,
  0.8410400948185843,
  0.40821177479355963,
  0.3175670517822356,
  1.4519254124185903,
  1.146802866198446,
  1.7211014558577302,
  0.44628880094602813,
  1.6107055479907009,
  0.3252778535993709,
  1.0326284189359256,
  1.6989626406019382,
  0.590655464959631,
  0.9490634509032452,
  0.4797226886234768,
  1.2945165334298303,
  1.4142979678909897,
  1.4960574646312148,
  0.6496016855656319,
  1.2309988246227364,
  1.2514202499789753,
  1.6572770013014309,
  0.4490092908720145,
  1.791099618126241,
  0.4422009760066159,
  1.3106939495160546,
  1.3749728167261914,
  1.597540268599413,
  1.544035257554224,
  1.0925884758883

In [405]:
dayPredict = np.array(dayPredict)
data_y = dayPredict[:, (gene_per_pop-1)]
fitness = fitness_func(pop_size, day_predict, yPred, gene_per_pop)
fitness

[2.0993481523684747,
 2.1755756408871876,
 2.950936282128967,
 3.106044684646755,
 2.786073227385031,
 4.133165701430577,
 2.721971794618448,
 2.1664332461187996,
 3.2134945846473815,
 3.473326778091496,
 3.0958761192920172,
 2.548015078035869,
 2.212097175495129,
 2.2035142679002817,
 2.702524749411075,
 3.7575446519756865,
 2.3583591883886386,
 2.648244107614451,
 1.463930281794112,
 2.854032923843326,
 1.8901441218517887,
 2.016993022717006,
 2.020490649243543,
 2.84897460019221,
 2.066198447081649,
 2.4556453722108547,
 2.5495884764501757,
 2.0887213230817783,
 2.5907441547445016,
 2.3221517160686656,
 2.567713875337441,
 2.24555993352834,
 1.964701864733175,
 2.307046280852706,
 1.868349236290682,
 3.124767517853678,
 2.2370651318495853,
 2.180310146715591,
 2.9064660300480574,
 2.5058965243136657,
 3.3556955699172675,
 2.1640206742751444,
 2.0397616092275697,
 2.579946621083882,
 1.9609749493041697,
 1.9669966568000532,
 3.2795001283254117,
 3.08251212068856,
 2.5949992154277894,

In [406]:
yProb = probability(fitness)
yProb

[1.661650365383437,
 1.7219850145012976,
 2.335688983216442,
 2.4584584883254195,
 2.2051985951193425,
 3.2714327493627495,
 2.1544618132958866,
 1.7147487380455642,
 2.543505918593249,
 2.749165117467763,
 2.450409989881718,
 2.0167737212353893,
 1.7508920927566045,
 1.7440986547435051,
 2.139069325995482,
 2.9741257717810345,
 1.8666596117800809,
 2.096105776489485,
 1.1587121863968506,
 2.2589892226166786,
 1.4960637506261127,
 1.5964656407239395,
 1.5992340392810052,
 2.2549855201656954,
 1.6354121162202084,
 1.9436623817649288,
 2.01801907829876,
 1.6532391474788772,
 2.050594117268936,
 1.8380011162645957,
 2.032365472273369,
 1.7773781255996521,
 1.5550767831053847,
 1.8260450469877791,
 1.4788129294532428,
 2.4732777562034975,
 1.7706544241033388,
 1.725732412630951,
 2.3004904333543954,
 1.9834365588867038,
 2.6560591027161453,
 1.7128391686961446,
 1.6144871537597165,
 2.0420477855267873,
 1.55212690060144,
 1.5568931288468495,
 2.595749818989757,
 2.439832281207068,
 2.05396

In [407]:
yRanges = rangePc(yProb)
yRanges

array([[  0.        ,   1.66165037],
       [  1.66165037,   3.38363538],
       [  3.38363538,   5.71932436],
       [  5.71932436,   8.17778285],
       [  8.17778285,  10.38298145],
       [ 10.38298145,  13.6544142 ],
       [ 13.6544142 ,  15.80887601],
       [ 15.80887601,  17.52362475],
       [ 17.52362475,  20.06713067],
       [ 20.06713067,  22.81629578],
       [ 22.81629578,  25.26670577],
       [ 25.26670577,  27.28347949],
       [ 27.28347949,  29.03437159],
       [ 29.03437159,  30.77847024],
       [ 30.77847024,  32.91753957],
       [ 32.91753957,  35.89166534],
       [ 35.89166534,  37.75832495],
       [ 37.75832495,  39.85443073],
       [ 39.85443073,  41.01314291],
       [ 41.01314291,  43.27213214],
       [ 43.27213214,  44.76819589],
       [ 44.76819589,  46.36466153],
       [ 46.36466153,  47.96389557],
       [ 47.96389557,  50.21888109],
       [ 50.21888109,  51.8542932 ],
       [ 51.8542932 ,  53.79795559],
       [ 53.79795559,  55.81597466],
 

In [408]:
roundWheel = round_wheel(pc,fitness,yRanges)
roundWheel

[27,
 23,
 48,
 10,
 11,
 23,
 22,
 35,
 2,
 44,
 27,
 25,
 22,
 10,
 18,
 7,
 1,
 0,
 2,
 4,
 18,
 41,
 5,
 49,
 19,
 12,
 33,
 5,
 7,
 22,
 32,
 30,
 24,
 32,
 2]